In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols 

In [2]:
spx = pd.read_csv('..\data\$spx_intraday-30min.csv', parse_dates = ['Time'])
spx['date'] = pd.to_datetime(spx['Time'].dt.date)
spx = spx.drop_duplicates('date')
spx['log_ret'] = np.log(spx['Last']) - np.log(spx['Last'].shift(1))
spx = spx[['date', 'log_ret','Volume']]
spx.dropna(inplace = True)

In [3]:
lstm_bal = pd.read_csv('../output/results/LSTM_balanced_results.csv',index_col=0)
lstm_bal['positive'] = (lstm_bal['outcome'] == 1)
lstm_bal['negative'] = (lstm_bal['outcome'] == -1)

In [4]:
lstm_bal['positive'].sum()+lstm_bal['negative'].sum() == len(lstm_bal)

# problem - no neutral tweets for control ?

True

In [5]:
lstm_bal['date'] = pd.to_datetime(lstm_bal['date'])
lstm_bal = lstm_bal[['date','positive','negative']]

In [6]:
multiple_tweets_dates = lstm_bal.groupby('date').sum()[(lstm_bal.groupby('date').sum()['negative'] > 1)|
                                                       (lstm_bal.groupby('date').sum()['positive'] > 1)]

mask = ~np.in1d(lstm_bal['date'], multiple_tweets_dates.index)

In [7]:
# only 99 days with single tweet. Does this make sense?
len(multiple_tweets_dates), mask.sum()

(1164, 99)

In [8]:
lstm_bal = lstm_bal.set_index('date').loc[mask] * 1
lstm_bal.reset_index(inplace = True)
lstm_bal = lstm_bal.drop_duplicates('date')

In [9]:
merged = lstm_bal.merge(spx, how = 'outer', on = 'date').fillna(0)

In [10]:
fit = ols('log_ret ~ C(positive)', data=merged).fit() 

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                log_ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                 0.0005430
Date:                Sun, 13 Dec 2020   Prob (F-statistic):              0.981
Time:                        01:22:35   Log-Likelihood:                 2642.7
No. Observations:                 774   AIC:                            -5281.
Df Residuals:                     772   BIC:                            -5272.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0005      0.000     -1.557      0.120      -0.001       0.000
C(positive)[T.1.0] -3.305e-05      0.001     -0.023      0.981      -0.003       0.003
==============================================================================
Omnibus:                      156.017   Durbin-Watson:                   2.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1102.430
Skew:                           0.707   Prob(JB):                    4.08e-240
Kurtosis:                       8.673   Cond. No.                         4.96
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""